In [ ]:
import os, types, pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from botocore.client import Config
import ibm_boto3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def __iter__(self): return 0

# 🔐 IBM Cloud Object Storage credentials (replace if needed)
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='1kngL5OrLamvz4N6tlqLZp4_Z1JIlHvNvzYFt3wJCc7h',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.au-syd.cloud-object-storage.appdomain.cloud')

bucket = 'nsappredictionproject-donotdelete-pr-i1xktoke41x2pp'
object_key = 'DistrictwisePensiondataundertheNationalSocialAssistanceProgrammeNSAP.csv'
body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType(__iter__, body)

df = pd.read_csv(body)
print("✅ Dataset loaded successfully\n📊 Shape:", df.shape, "\n🧾 Columns:", df.columns.tolist())

df.dropna(inplace=True)
label_encoders = {}
for col in df.select_dtypes(include='object').columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

X = df.drop('schemecode', axis=1)
y = df['schemecode']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("\n🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))
print("\n📌 Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

importances = model.feature_importances_
feat_names = X.columns
feat_imp = pd.Series(importances, index=feat_names).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=feat_imp.values, y=feat_imp.index, palette='Blues_d')
plt.title("🔍 Feature Importance")
plt.xlabel("Importance Score")
plt.ylabel("Features")
plt.tight_layout()
plt.show()
